In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from matplotlib import pyplot as plt
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['mathtext.default'] = 'regular'  
plt.rcParams['mathtext.fontset'] = 'custom'    
plt.rcParams['mathtext.rm'] = 'Arial'          
plt.rcParams['mathtext.it'] = 'Arial:italic'  
plt.rcParams['mathtext.bf'] = 'Arial:bold'     
data = pd.read_csv('a-alldatafinal.csv')

X = data[['Zeta potential', 'Pore Radius', '压力/bar', '盐浓度/ppm']]
y = data['NaCl']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = xgb.XGBRegressor(
    learning_rate=0.05,
    max_depth=4,
    subsample=0.6,
    colsample_bytree=0.2,
    n_estimators=1000,
    random_state=42
)
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_pred = model.predict(X_test)

r_squared = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'Fold {fold + 1}:')
print(f'均方根误差RMSE: {rmse}')
print(f'平均绝对误差MAE: {mae}')
print(f'决定系数R^2: {r_squared}')
print(f'MSE: {mse}')
print('-' * 30)

fig1 = plt.figure(figsize=(10, 10))
plt.xlabel('Test values', fontsize=28, fontweight='bold', color='black')
plt.ylabel('Prediction values', fontsize=28, fontweight='bold', color='black')
axes = plt.gca()
axes = plt.gca()
x_axis = axes.spines['bottom']
y_axis = axes.spines['left']
top_axis = axes.spines['top']
right_axis = axes.spines['right']

line_color = 'black'
line_width = 2

x_axis.set_color(line_color)
x_axis.set_linewidth(line_width)
y_axis.set_color(line_color)
y_axis.set_linewidth(line_width)
top_axis.set_color(line_color)
top_axis.set_linewidth(line_width)
right_axis.set_color(line_color)
right_axis.set_linewidth(line_width)

plt.scatter(y_train, y_train_pred, color='blue', label='Train(80%) ', marker='o', s=80, linewidths=3,
            facecolors='none', edgecolors='blue')

plt.scatter(y_test, y_pred, color='red', label='Test(20%)', marker='s', s=80, linewidths=3.5,
            facecolors='none', edgecolors='red')

#plt.title('XGboost', fontsize=30, fontweight='bold', color='black')
plt.plot([min(y_train.min(), y_test.min()), max(y_train.max(), y_test.max())],
             [min(y_train.min(), y_test.min()), max(y_train.max(), y_test.max())],
             color='black', linestyle='-', linewidth=2, label='x=y')
plt.xticks(fontsize=26)
plt.yticks(fontsize=26)
plt.legend(fontsize='x-large', prop={'size': 22})
for spine in plt.gca().spines.values():
    spine.set_linewidth(2)
#plt.text(-0.2, 0.97, '(g)', fontsize=34,transform=plt.gca().transAxes, color='black')
plt.show()

#稳定性验证部分
results = []

# ====== 用随机种子 1~300 做300次训练测试 ======
n_trials =500
test_size = 30  

for seed in range(1, n_trials + 1):
    np.random.seed(seed)
    all_indices = np.arange(len(data))
    test_indices = np.random.choice(all_indices, size=test_size, replace=False)
    train_indices = np.setdiff1d(all_indices, test_indices)

    X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)

    results.append({
        'seed': seed,
        'r2': r2,
        'mae': mae,
        'mse': mse,
        'rmse': rmse
    })

results_df = pd.DataFrame(results)

# ====== 选取 r² 排名前50的结果 ======
top50 = results_df.sort_values('r2', ascending=False).head(50)

# ====== 统计这50次的平均值 ± 标准差 ======
top50_to_save = top50[['seed', 'r2', 'rmse', 'mae']]


model.fit(X, y)

feature_importance = model.feature_importances_
feature_names = X.columns
feature_importance_dict = dict(zip(feature_names, feature_importance))
sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

sorted_feature_names, sorted_importance = zip(*sorted_feature_importance)
for feature, importance in sorted_feature_importance:
    print(f"{feature}: {importance}")
